In [1]:
from BayesianNetworks import *

In [2]:
BatteryState = readFactorTable(['battery'], [0.9, 0.1], [[1, 0]])
FuelState = readFactorTable(['fuel'], [0.9, 0.1], [[1, 0]])
GaugeBF = readFactorTable(['gauge', 'battery', 'fuel'],
                          [0.8, 0.2, 0.2, 0.1, 0.2, 0.8, 0.8, 0.9],
                          [[1, 0], [1, 0], [1, 0]])
carNet = [BatteryState, FuelState, GaugeBF]

In [3]:
def joinFactors(factor1, factor2):
    fac1 = factor1.copy(deep=True)
    fac2 = factor2.copy(deep=True)
    # your code
    common = list(fac1.columns.intersection(fac2.columns))
    if common == ['probs']:  #无公共因素
        fac1['key'] = [1] * fac1.shape[0]
        fac2['key'] = [1] * fac2.shape[0]

        factor = pd.merge(fac1, fac2, on='key', how='outer')
        factor['probs'] = factor['probs_x'] * factor['probs_y']
        factor = factor.drop(['probs_x', 'probs_y', 'key'], axis=1)
    else:  #有公共因素
        common.remove('probs')
        factor = pd.merge(fac1, fac2, on=common, how='outer')  #使用公共因素连接
        factor['probs'] = factor['probs_x'] * factor['probs_y']
        factor = factor.drop(['probs_x', 'probs_y'], axis=1)
    return factor

In [4]:
def marginalizeFactor(factorTable, hiddenVar):
    # your code
    #区分参数类型
    if type(hiddenVar) == str:
        common = list(set([hiddenVar]).intersection(factorTable.columns))
    elif type(hiddenVar) == list:
        common = list(set(hiddenVar).intersection(factorTable.columns))
    else:
        common = []
    if common:  #hiddenVar出现在factorTable中
        result = list(factorTable.columns)
        for i in common:
            result.remove(i)
        result.remove('probs')
        if not result:
            return []
        factor = factorTable.drop(common, axis=1)
        factor = factor.groupby(result).sum()  #剩余相同状态的概率相加
        factor = factor.reset_index()  #旧索引将添加为列，并使用新的顺序索引
        return factor
    else:  #factorTable不含hiddenVar
        return factorTable


def marginalizeNetworkVariables(bayesNet, hiddenVar):
    # your code
    Net = []
    for table in bayesNet:
        Net.append(marginalizeFactor(table, hiddenVar))
    return Net

In [5]:
def evidenceUpdateNet(bayesNet, evidenceVars, evidenceVals):
    # your code
    Net = bayesNet.copy()
    if type(evidenceVars) == str:
        evidenceVars = [evidenceVars]
    if type(evidenceVars) == list:
        num = len(evidenceVars)
    for i in range(len(Net)):
        for j in range(num):
            if evidenceVars[j] in Net[i].columns:
                Net[i] = Net[i][Net[i][evidenceVars[j]].isin([evidenceVals[j]
                                                              ])]
                Net[i] = Net[i].reset_index(drop=True)  #index修改
    return Net


def inference(bayesNet, hiddenVar, evidenceVars, evidenceVals):
    # your code
    # 使用evidenceVars
    Net = evidenceUpdateNet(bayesNet, evidenceVars, evidenceVals)
    # 使用hiddenVar
    while hiddenVar:
        hid = hiddenVar.pop()
        Net_hid = [x for x in Net if hid in x.columns]
        Net = [x for x in Net if hid not in x.columns]
        if Net_hid:
            tmp = Net_hid[0]
        for table in Net_hid[1:]:
            tmp = joinFactors(tmp, table)
        Net.append(tmp)
        Net = marginalizeNetworkVariables(Net, hid)
    # 剩余全部相乘
    if Net:
        factor = Net[0]
    for table in Net[1:]:
        factor = joinFactors(factor, table)
    # Normalize
    factor['probs'] = factor['probs'] / factor['probs'].sum()
    return factor

In [6]:
print("inference starts")
print(inference(carNet, ['battery', 'fuel'], [],
                []))  ## chapter 8 equation (8.30)
print(inference(carNet, ['battery'], ['fuel'],
                [0]))  ## chapter 8 equation (8.31)
print(inference(carNet, ['battery'], ['gauge'],
                [0]))  ##chapter 8 equation  (8.32)
print(inference(carNet, [], ['gauge', 'battery'],
                [0, 0]))  ## chapter 8 equation (8.33)
print("inference ends")

inference starts
   gauge  probs
0      0  0.315
1      1  0.685
   fuel  gauge  probs
0     0      0   0.81
1     0      1   0.19
   fuel  gauge     probs
0     1      0  0.742857
1     0      0  0.257143
   battery  fuel  gauge     probs
0        0     1      0  0.888889
1        0     0      0  0.111111
inference ends


In [7]:
marginalizeFactor(carNet[2], 'battery')

,fuel,gauge,probs
0,0,0,1.7
1,0,1,0.3
2,1,0,1.0
3,1,1,1.0


In [8]:
marginalizeFactor(joinFactors(GaugeBF, BatteryState), ['gauge', 'battery'])

,fuel,probs
0,0,1.0
1,1,1.0


In [9]:
# marginalizeNetworkVariables(carNet, 'fuel') ## this returns back a list
marginalizeNetworkVariables(carNet, ['battery', 'fuel'])

[[],
 [],
    gauge  probs
 0      0    2.7
 1      1    1.3]